# Linear models, loss functions, gradients, SGD
(c) Deniz Yuret, 2019

* Objectives: Define, train and visualize a simple model; understand gradients and SGD; learn to use the GPU.
* Prerequisites: [Callable objects](https://docs.julialang.org/en/v1/manual/methods/#Function-like-objects-1), [Generator expressions](https://docs.julialang.org/en/v1/manual/arrays/#Generator-Expressions-1), [MNIST](02.mnist.ipynb)
* Knet: [dir](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.dir), [Data](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.minibatch), [mnistdata](https://github.com/denizyuret/Knet.jl/blob/8fdb4c64322905a907e2be7f63d25fcd15175dfb/data/mnist.jl#L45) (mnist.jl)
* Knet: [accuracy](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.accuracy), [zeroone](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.zeroone), [nll](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.nll), [sgd](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.sgd)
* AutoGrad: [Param, @diff, value, params, grad](http://denizyuret.github.io/Knet.jl/latest/reference.html#AutoGrad)
* Knet: [progress, progress!](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.progress), [gpu](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.gpu), [KnetArray](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.KnetArray), [load](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.load), [save](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.save)

In [ ]:
# Set display width, load packages, import symbols
ENV["COLUMNS"]=72
using Pkg; for p in ("Knet","AutoGrad","Plots","Images","ImageMagick"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Knet: Knet, dir, Data, nll, Param, @diff, value, params, grad, progress, progress!, gpu, KnetArray, load, save

In [ ]:
# Load data (mnistdata basically replicates mnist.ipynb)
include(Knet.dir("data","mnist.jl"))
dtrn,dtst = mnistdata(xsize=(784,:),xtype=Array)
println.(summary.((dtrn,dtst)));

## Define linear model

In [ ]:
# In Julia we define a new datatype using `struct`:
struct Linear; w; b; end

# The new struct comes with a default constructor:
model = Linear(0.01 * randn(10,784), zeros(10))

# We can define other constructors with different inputs:
Linear(i::Int,o::Int,scale=0.01) = Linear(scale * randn(o,i), zeros(o))

# This one allows instances to be defined using input and output sizes:
model = Linear(784,10)

## Prediction and accuracy

In [ ]:
# We turn Linear instances into callable objects for prediction:
(m::Linear)(x) = m.w * x .+ m.b

In [ ]:
# Let's take the first minibatch from the test set
x,y = first(dtst)
summary.((x,y))

In [ ]:
# correct answers are given as an array of integers (remember we use 10 for 0)
Int.(y)'

In [ ]:
# Display predictions on the first minibatch: a 10x100 score matrix
ypred = model(x)

In [ ]:
# We can calculate the accuracy of our model for the first minibatch
using Statistics: mean
accuracy(model,x,y) = mean(y' .== map(i->i[1], findmax(Array(model(x)),dims=1)[2]))
accuracy(model,x,y)

In [ ]:
# We can calculate the accuracy of our model for the whole test set
accuracy(model,data) = mean(accuracy(model,x,y) for (x,y) in data)
accuracy(model,dtst)

In [ ]:
# ZeroOne loss (or error) is defined as 1 - accuracy
zeroone(x...) = 1 - accuracy(x...)
zeroone(model,dtst)

## Negative log likelihood

In [ ]:
# With two inputs, let the model compute a loss. For classification we use
# negative log likelihood (aka cross entropy, softmax loss, NLL)
function (m::Linear)(x, y)
    scores = m(x)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    answerprobs = (probabilities[y[i],i] for i in 1:length(y))
    mean(-log.(answerprobs))
end

In [ ]:
# Calculate loss of our model for the first minibatch
model(x,y)

In [ ]:
# We can also use the Knet nll implementation for efficiency
(m::Linear)(x, y) = nll(m(x), y)
model(x,y)

In [ ]:
# If the input is a dataset compute average loss:
(m::Linear)(data::Data) = mean(m(x,y) for (x,y) in data)

In [ ]:
# Here is per-instance average negative log likelihood for the whole test set
model(dtst)

## Calculating the gradient using AutoGrad

In [ ]:
import AutoGrad
@doc AutoGrad

In [ ]:
# Redefine the constructor to use Param's so we can compute gradients
Linear(i::Int,o::Int,scale=0.01) = 
    Linear(Param(scale * randn(o,i)), Param(zeros(o)))

In [ ]:
# Set random seed for replicability
using Random; Random.seed!(9);

In [ ]:
# Use a larger scale to get a large initial loss
model = Linear(784,10,1.0)

In [ ]:
# We can still do predictions and calculate loss:
model(x,y)

In [ ]:
# And we can do the same loss calculation also computing gradients:
J = @diff model(x,y)

In [ ]:
# To get the actual loss value from J:
value(J)

In [ ]:
# params(J) returns an iterator of Params J depends on (i.e. model.b, model.w):
params(J) |> collect

In [ ]:
# To get the gradient of a parameter from J:
∇w = grad(J,model.w)

In [ ]:
# Note that each gradient has the same size and shape as the corresponding parameter:
@show ∇b = grad(J,model.b);

## Checking the gradient using numerical approximation

What does ∇b represent?

∇b[10] = 0.79 means if I increase b[10] by ϵ, loss will increase by 0.79ϵ

In [ ]:
# Loss for the first minibatch with the original parameters
@show value(model.b)
model(x,y)

In [ ]:
# To numerically check the gradient let's increase the last entry of b by +0.1.
model.b[10] = 0.1

In [ ]:
# We see that the loss moves by ≈ +0.79*0.1 as expected.
@show value(model.b)
model(x,y)

In [ ]:
# Reset the change.
model.b[10] = 0

## Checking the gradient using manual implementation

In [ ]:
# Without AutoGrad we would have to define the gradients manually:
function nllgrad(model,x,y)
    scores = model(x)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    for i in 1:length(y); probabilities[y[i],i] -= 1; end
    dJds = probabilities / length(y)
    dJdw = dJds * x'
    dJdb = vec(sum(dJds,dims=2))
    dJdw,dJdb
end;

In [ ]:
∇w2,∇b2 = nllgrad(model,x,y)

In [ ]:
∇w2 ≈ ∇w

In [ ]:
∇b2 ≈ ∇b

## Training with Stochastic Gradient Descent (SGD)

In [ ]:
# Here is a single SGD update:
function sgdupdate!(func, args; lr=0.1)
    fval = @diff func(args...)
    for param in params(fval)
        ∇param = grad(fval, param)
        param .-= lr * ∇param
    end
    return value(fval)
end

In [ ]:
# We define SGD for a dataset as an iterator so that:
# 1. We can monitor and report the training loss
# 2. We can take snapshots of the model during training
# 3. We can pause/terminate training when necessary
sgdtrain(func, data; lr=0.1) = 
    (sgdupdate!(func, args; lr=lr) for args in data)

In [ ]:
# Let's train a model for 10 epochs to compare training speed on cpu vs gpu.
# progress!(itr) displays a progress bar when wrapped around an iterator.
model = Linear(784,10)
@show model(dtst)
progress!(sgdtrain(model, repeat(dtrn,10)))
@show model(dtst);

## Using the GPU

In [ ]:
# To work on the GPU, all we have to do is convert Arrays to KnetArrays:
if gpu() >= 0  # gpu() returns a device id >= 0 if there is a GPU, -1 otherwise
    atype = KnetArray{Float32}
    dtrn,dtst = mnistdata(xsize=(784,:),xtype=atype)
    Linear(i::Int,o::Int,scale=0.01) = 
        Linear(Param(atype(scale * randn(o,i))), 
               Param(atype(zeros(o))))

    model = Linear(784,10)
    @show model(dtst)
    progress!(sgdtrain(model,repeat(dtrn,10)))
    @show model(dtst)
end;


## Recording progress

In [ ]:
using Base.Iterators: flatten

if (print("Train from scratch? (~77s) "); readline()[1]=='y')
    model = Linear(784,10)
    # This will take every nth element of an iterator:
    takeevery(n,itr) = (x for (i,x) in enumerate(itr) if i % n == 1)
    # We will use it to snapshot model and results every epoch (i.e. 600 iterations)
    # (progress returns an iterator, progress! returns nothing)
    lin = ((deepcopy(model),model(dtrn),model(dtst),zeroone(model,dtrn),zeroone(model,dtst))
           for x in takeevery(length(dtrn), progress(sgdtrain(model,repeat(dtrn,100)))))
    # Save it as a 5x100 array
    lin = reshape(collect(flatten(lin)),(5,:))
    save("lin.jld2","results",lin)
else
    isfile("lin.jld2") || download("http://people.csail.mit.edu/deniz/models/tutorial/lin.jld2","lin.jld2")
    lin = load("lin.jld2","results")    
end;

## Linear model shows underfitting

In [ ]:
using Plots; default(fmt = :png)

In [ ]:
# Demonstrates underfitting: training loss not close to 0
# Also slight overfitting: test loss higher than train
plot([lin[2,:], lin[3,:]],ylim=(.0,.4),labels=[:trnloss :tstloss],xlabel="Epochs",ylabel="Loss")

In [ ]:
# this is the error plot, we get to about 7.5% test error, i.e. 92.5% accuracy
plot([lin[4,:], lin[5,:]],ylim=(.0,.12),labels=[:trnerr :tsterr],xlabel="Epochs",ylabel="Error")

## Visualizing the learned weights

In [ ]:
# Let us visualize the evolution of the weight matrix as images below
# Each row is turned into a 28x28 image with positive weights light and negative weights dark gray
using Images, ImageMagick
for t in 10 .^ range(0,stop=log10(size(lin,2)),length=20) #logspace(0,2,20)
    i = floor(Int,t)
    f = lin[1,i]
    w1 = reshape(Array(value(f.w))', (28,28,1,10))
    w2 = clamp.(w1.+0.5,0,1)
    IJulia.clear_output(true)
    display(hcat([mnistview(w2,i) for i=1:10]...))
    display("Epoch $i")
    sleep(1) # (0.96^i)
end